### <u>**Tutorial**</u>
---

1. **Access Yahoo Finance Screener**  
   - Go to the [Yahoo Finance Screener](https://finance.yahoo.com/research-hub/screener/most_actives/).

2. **Open Developer Tools**  
   - Right-click on the webpage and select **Inspect** to launch your browser’s developer tools.

3. **Monitor Network Requests**  
   - Select the **Network** tab in the developer tools.  
   - Filter by **Fetch/XHR** to view relevant requests.

4. **Trigger Data Retrieval**  
   - Click the button to proceed to the next page of results.  
   - Keep an eye on the network requests being made.

5. **Identify Target Request**  
   - Look for the request starting with:  
     ```
     https://query1.finance.yahoo.com/v1/finance/screener/
     ```

6. **Copy Request as cURL**  
   - Right-click on that request and select **Copy as cURL (bash)**.

7. **Convert cURL to Python**  
   - Use a [cURL to Python converter](https://curlconverter.com/python/).  
   - Paste your cURL command there to obtain Python code.

8. **Extract Necessary Values**  
   - From the generated Python code, note the following values:  
     - `GUC`  
     - `A1S`  
     - `EuConsent`  
     - `A1`  
     - `A3`  
     - `cmp`  
     - `User-Agent`  
     - `x-crumb`

9. **Update Configuration**  
   
   - Option 1:
       - Within the `Utils` folder, open the `Keys` file.  
       - Replace the default placeholders with these extracted values.  
       - Save your changes.
   
   - Option 2:
      - Use builtin functions as showed below.

In [1]:
from F4 import EQTickers

In [6]:
#0. Check the default keys values:
EQTickers.GetKeys()

{'Your_GUC_Value': 'insert_your_value',
 'Your_A1S_Value': 'insert_your_value',
 'Your_EuConsent_Value': 'insert_your_value',
 'Your_A1_Value': 'insert_your_value',
 'Your_A3_Value': 'insert_your_value',
 'Your_cmp_Value': 'insert_your_value',
 'Your_User_Agent': 'insert_your_value',
 'Your_x_crumb_Value': 'insert_your_value'}

In [7]:
#1. Replace the values with your own keys:
your_keys = {'Your_GUC_Value': "Definitely_not_a_real_key",
'Your_A1S_Value' : "Definitely_not_a_real_key",
'Your_EuConsent_Value' : "Definitely_not_a_real_key",
'Your_A1_Value' : "Definitely_not_a_real_key",
'Your_A3_Value' : "Definitely_not_a_real_key",
'Your_cmp_Value' : "Definitely_not_a_real_key",
'Your_User_Agent' : "Definitely_not_a_real_key",
'Your_x_crumb_Value' : "Definitely_not_a_real_key"}

#2. Set the keys:
EQTickers.SetKeys(your_keys) #Depending on which environement you are using, you may need to restart the kernel for the changes to take effect.

Keys updated successfully.


In [8]:
#3. Check if the keys are set:
EQTickers.GetKeys()

{'Your_GUC_Value': 'Definitely_not_a_real_key',
 'Your_A1S_Value': 'Definitely_not_a_real_key',
 'Your_EuConsent_Value': 'Definitely_not_a_real_key',
 'Your_A1_Value': 'Definitely_not_a_real_key',
 'Your_A3_Value': 'Definitely_not_a_real_key',
 'Your_cmp_Value': 'Definitely_not_a_real_key',
 'Your_User_Agent': 'Definitely_not_a_real_key',
 'Your_x_crumb_Value': 'Definitely_not_a_real_key'}

In [5]:
#4. If you want to reset the keys to the default values:

EQTickers.ResetKeys() #Depending on which environement you are using, you may need to restart the kernel for the changes to take effect.

Reset done successfully. 8 keys were updated.


In [9]:
#5. Check if the keys are reset:
EQTickers.GetKeys()

{'Your_GUC_Value': 'Definitely_not_a_real_key',
 'Your_A1S_Value': 'Definitely_not_a_real_key',
 'Your_EuConsent_Value': 'Definitely_not_a_real_key',
 'Your_A1_Value': 'Definitely_not_a_real_key',
 'Your_A3_Value': 'Definitely_not_a_real_key',
 'Your_cmp_Value': 'Definitely_not_a_real_key',
 'Your_User_Agent': 'Definitely_not_a_real_key',
 'Your_x_crumb_Value': 'Definitely_not_a_real_key'}

Once you've set your keys, you'll finally be able to use the script.

In [9]:
# Check the available regions:
EQTickers.AvailableRegions()

['ar',
 'at',
 'be',
 'br',
 'ca',
 'ch',
 'cl',
 'cn',
 'cz',
 'de',
 'dk',
 'fr',
 'gb',
 'hk',
 'hu',
 'id',
 'il',
 'in',
 'is',
 'it',
 'jp',
 'kr',
 'kw',
 'mx',
 'nl',
 'no',
 'pl',
 'sa',
 'se',
 'tr',
 'tw',
 'us',
 've',
 'za']

In [17]:
# Check the available infomations:
EQTickers.FetchedInformations()

['Ask',
 'Ask Size',
 'Average Analyst Rating',
 'Average Daily Volume 10 Day',
 'Average Daily Volume 3 Month',
 'Bid',
 'Bid Size',
 'Book Value',
 'Corporate Actions',
 'Country',
 'Crypto Tradeable',
 'Currency',
 'Custom Price Alert Confidence',
 'Display Type',
 'Dividend Rate',
 'Dividend Yield',
 'EPS Current Year',
 'EPS Forward',
 'EPS Trailing Twelve Months',
 'ESG Populated',
 'Earnings Call Timestamp End',
 'Earnings Call Timestamp Start',
 'Earnings Timestamp',
 'Earnings Timestamp End',
 'Earnings Timestamp Start',
 'Exchange',
 'Exchange Data Delayed By',
 'Exchange Timezone Name',
 'Exchange Timezone Short Name',
 'Fifty Day Average',
 'Fifty Day Average Change',
 'Fifty Day Average Change Percent',
 'Fifty Two Week Change Percent',
 'Fifty Two Week High',
 'Fifty Two Week High Change',
 'Fifty Two Week High Change Percent',
 'Fifty Two Week Low',
 'Fifty Two Week Low Change',
 'Fifty Two Week Low Change Percent',
 'Fifty Two Week Range',
 'Financial Currency',
 'First

In [4]:
# Get tickers for a list of regions:
EQTickers.FetchRegionsTickers(['fr'], BatchSize=750, WaitTime=1)

Fetching tickers for the following regions: ['fr']
| Region:         fr | Total count received:    642 | Total tickers retrieved:    642 | Current Offset:   600 |


Processing Batches  : 100%|██████████████████████████████| 1/1 [00:02<00:00,  2.61s/Batch]


,Ticker,Country,Sector,Industry,Language,Region,Quote Type,Display Type,Quote Source Name,Triggerable,...,Tradeable,Crypto Tradeable,Trailing PE,Dividend Rate,Dividend Yield,Average Analyst Rating,Previous Name,Name Change Date,Dividend Date,IPO Expected Date
0,ATO.PA,France,Technology,Information Technology Services,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALVER.PA,France,Utilities,Utilities—Independent Power Producers,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ORA.PA,France,Communication Services,Telecom Services,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,13.272728,0.72,7.09,1.5 - Buy,NaN,NaN,NaN,NaN
3,VIV.PA,France,Communication Services,Entertainment,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,6.700000,0.25,9.72,2.5 - Buy,Vivendi SA,2025-01-17,NaN,NaN
4,ENGI.PA,France,Utilities,Utilities—Diversified,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,7.960396,1.43,9.03,1.6 - Buy,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,MLIMP.PA,France,Communication Services,Publishing,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,6.516854,0.40,6.25,NaN,NaN,NaN,NaN,NaN
638,ALLLN.PA,France,Communication Services,Telecom Services,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,MLHMC.PA,France,Consumer Cyclical,Lodging,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,NaN,416.00,7.50,NaN,NaN,NaN,NaN,NaN
640,ALIKO.PA,France,Healthcare,Medical Devices,en-US,US,EQUITY,Equity,Delayed Quote,False,...,False,False,NaN,NaN,NaN,NaN,Ikonisys,2025-01-17,NaN,NaN


In [5]:
#Get tickers for all regions:
EQTickers.FetchAllTickers(BatchSize=1000, WaitTime=1)

Fetching tickers for the following regions: ['ar', 'at', 'be', 'br', 'ca', 'ch', 'cl', 'cn', 'cz', 'de', 'dk', 'fr', 'gb', 'hk', 'hu', 'id', 'il', 'in', 'is', 'it', 'jp', 'kr', 'kw', 'mx', 'nl', 'no', 'pl', 'sa', 'se', 'tr', 'tw', 'us', 've', 'za']
| Region:         ar | Total count received:    244 | Total tickers retrieved:    244 | Current Offset:   200 |
| Region:         at | Total count received:    835 | Total tickers retrieved:    833 | Current Offset:   800 |
| Region:         be | Total count received:    115 | Total tickers retrieved:    115 | Current Offset:   100 |
| Region:         br | Total count received:   1182 | Total tickers retrieved:   1181 | Current Offset:  1100 |
| Region:         ca | Total count received:   2900 | Total tickers retrieved:   2899 | Current Offset:  2900 |
| Region:         ch | Total count received:    227 | Total tickers retrieved:    227 | Current Offset:   200 |
| Region:         cl | Total count received:    143 | Total tickers retrieved: 

: 

Here I voluntarily interrupted the execution of the code to avoid a long output. You can run the code to get the tickers for all regions. Otherwise, you can find a .zip file in the repository which contains a version of the complete file as of January 17, 2025.